<a href="https://colab.research.google.com/github/reknahs/email-anonymization/blob/main/azure_anonymization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.4/327.4 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 10.7 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
%%capture
from google.colab import userdata
API_VERSION = userdata.get('API_VERSION')
API_KEY = userdata.get('API_KEY')
AZURE_ENDPOINT = userdata.get('AZURE_ENDPOINT')
DEPLOYMENT_NAME = userdata.get('DEPLOYMENT_NAME')

In [ ]:
# Function to count tokens using tiktoken
def count_tokens(prompt: str, model: str = "gpt-3.5-turbo") -> int:
    encoding = tiktoken.encoding_for_model(model)
    tokens = encoding.encode(prompt)
    return len(tokens)

In [ ]:
import email
from email import policy
from email.parser import BytesParser
import time

beg = time.time()

# Path to your .eml file
eml_file_path = '/content/drive/MyDrive/emails/emailreturn1.eml'
# eml_file_path = '/content/drive/MyDrive/emails/emailreturn2.eml'
# eml_file_path = '/content/drive/MyDrive/emails/emailreturn3.eml'
# eml_file_path = '/content/drive/MyDrive/emails/emailreturn4.eml'

# Read the .eml file
with open(eml_file_path, 'rb') as f:
    msg = BytesParser(policy=policy.default).parse(f)

# Extract subject
subject = msg['subject']
print('Subject:', subject)

# Extract body (both plain text and HTML)
body = ""
if msg.is_multipart():
    for part in msg.iter_parts():
        content_type = part.get_content_type()
        content_disposition = str(part.get('Content-Disposition'))

        # Look for plain text parts
        if content_type == 'text/plain' and 'attachment' not in content_disposition:
            body = part.get_payload(decode=True).decode(part.get_content_charset())
            break
        # Look for HTML parts
        elif content_type == 'text/html' and 'attachment' not in content_disposition:
            body = part.get_payload(decode=True).decode(part.get_content_charset())
            break
else:
    # If the message isn't multipart, the payload is simple text
    body = msg.get_payload(decode=True).decode(msg.get_content_charset())

import os
from openai import AzureOpenAI
import tiktoken

client = AzureOpenAI(
    api_key=API_KEY,
    api_version=API_VERSION,
    azure_endpoint = AZURE_ENDPOINT
    )

deployment_name= DEPLOYMENT_NAME

# Send a completion call to generate an answer
prompt = body
content = '''
Your job is to output a given email but after swapping ONLY PII with fake info. PII is anything that can give away someone's identity.

For example, you should REPLACE ALL instances of the following for the CUSTOMER ONLY:
-Name
-Billing/shipping address
-Phone number
-Email address
-Order number
-Any portion of credit card number

Replace the above with fake names/addresses/phone numbers/emails/order numbers/credit card numbers/etc. Delete all links. Do not change dollar amounts.
Example:

Input:
Hi James,

We'd like to confirm your order (Order number: 6823) to the 8523 Main St. San Francisco, CA 92356.
Paid with Mastercard ending in 9823.

Example of your output:
Hi Stephen,

We'd like to confirm your order (Order number: 9866) to the 1323 Oak St. Orlando, FL 12235.
Paid with Mastercard ending in 0912.
'''
start = time.time()
response = client.chat.completions.create(
    model=deployment_name,
    messages=[{"role": "system", "content": content},
              {"role": "user", "content": prompt}
              ]
)
end = time.time()
print("API call: ", end-start)
print("Total: ", end-beg)
print("Tokens: ", count_tokens(content+prompt))
print(response.choices[0].message)

Subject: Subject: Flip It and Reverse It (Refund On The Way)
API call:  2.8960280418395996
Total:  3.389348268508911
Tokens:  527
ChatCompletionMessage(content="Hi Raj,\nWe've refunded your order for $110.00 USD. Heads up: while the refund is immediate on our end, it can take a few days for your bank to process.\nOrder 523242542\n1 Women's Wool Runners - Navy Blue (Cream Sole) - 9\t$110.00\nSubtotal\t$110.00\nFast Shipping (Delivered in 3-5 Business Days)\t$6.00\nTotal\t$116.00\n(Paid with Visa...7363)\t$116.00\nRefund to Visa...7363\t- $110.00\n\t\t\nHELP\tCONTACT US\t1.888.963.8944\n\t\t\t\n© 2023 Allbirds, Inc.\n730 Montgomery St. San Francisco, CA 94111", role='assistant', function_call=None, tool_calls=None)


In [ ]:
print(count_tokens(prompt))

323


In [ ]:
print(response.choices[0].message.content)

Hi Raj,
We've refunded your order for $110.00 USD. Heads up: while the refund is immediate on our end, it can take a few days for your bank to process.
Order 523242542
1 Women's Wool Runners - Navy Blue (Cream Sole) - 9	$110.00
Subtotal	$110.00
Fast Shipping (Delivered in 3-5 Business Days)	$6.00
Total	$116.00
(Paid with Visa...7363)	$116.00
Refund to Visa...7363	- $110.00
		
HELP	CONTACT US	1.888.963.8944
			
© 2023 Allbirds, Inc.
730 Montgomery St. San Francisco, CA 94111


In [ ]:
print(body)

Hi Rohan,
We've refunded your order for $110.00 USD. Heads up: while the refund is immediate on our end, it can take a few days for your bank to process.
Order 649594665 <https://www.allbirds.com/11044168/orders/5c3c2ad86013c7f694c91b6e754ca11e/authenticate?key=c8dd6330d211f5f96f23357eab2414e5?utm_source=email-t&amp;utm_medium=email&amp;utm_campaign=orefund> 
 <https://cdn.shopify.com/s/files/1/1104/4168/products/WR3MDPG_SHOE_ANGLE_GLOBAL_MENS_WOOL_RUNNER_DAPPLE_GREY_CREAM_81817d0e-7018-4c5c-9f46-8495e51b6cbd_100x80_crop_left.png?v=1679676212> 	1 Women's Wool Runners - Dapple Grey (Cream Sole) - 9	$110.00
Subtotal	$110.00
Fast Shipping (Delivered in 3-5 Business Days)	$6.00
Total	$116.00
(Paid with Visa...3584)	$116.00
Refund to Visa...3584	- $110.00
		
HELP	CONTACT US	1.888.963.8944
			
© 2023 Allbirds, Inc.
730 Montgomery St. San Francisco, CA 94111


